In [8]:
import pandas as pd
df=pd.read_excel("data.xlsx")
df[0:10]


,id,line,account,date_trans,date_entry,dc,amount,seed
0,1,1,18,2020-07-01,2020-07-04,C,-34278.37,0
1,1,2,6,2020-07-01,2020-07-04,D,26341.71,0
2,1,3,17,2020-07-01,2020-07-04,D,7936.66,0
3,2,1,16,2020-07-02,2020-07-04,C,-11525.70,0
4,2,2,1,2020-07-02,2020-07-04,D,11525.70,0
5,3,1,12,2020-07-04,2020-07-04,C,-19091.40,0
6,3,2,10,2020-07-04,2020-07-04,D,39214.51,0
7,4,1,19,2020-07-01,2020-07-05,C,-43888.82,0
8,4,2,18,2020-07-01,2020-07-05,D,45922.77,0
9,4,3,4,2020-07-01,2020-07-05,D,42045.58,0


In [9]:
field_to_check='amount'
list_to_check=df[field_to_check].to_list()
initial_positions=list(range(len(list_to_check)))
matched=[False for n in initial_positions]
matching_ref=[[] for n in initial_positions]
tuples=list(zip(df.index,df[field_to_check]))
debs_pos=[x[0] for x in tuples if x[1] > 0]
debs_amt=[x[1] for x in tuples if x[1] > 0]
creds_pos=[x[0] for x in tuples if x[1] < 0]
creds_amt=[x[1] for x in tuples if x[1] < 0]

In [10]:
def check_match_list():
    running=0
    for n in match_list:
        tot=0
        for m in n:
            for v in m:
                tot=tot+df.iloc[v]['amount']
        if round(tot,1)==0:
            running=running+1
    print(running,len(match_list))


In [11]:
def find_possible_matches(v,list_amounts,ref="", list_refs=[],amount_tolerance=0.005):
    matches=[]
    pos=[]
    ref=[]
    for i, w in enumerate(list_amounts):
        if abs(v-w) <= amount_tolerance:
            matches.append(w)
            pos.append(i)
            try:
                ref.append(list_refs[i])
            except:
                ref.append("")
    if len(matches)==1:
        return pos[0], matches[0], 1
    if len(matches)==0:
        return False, False, False
    if len(matches)>1:
        return pos[0], matches[0], len(matches)  
        #here we would develop something to choose, either least tolerance, or first/last 
        # or using regexp to select based on a potential reference field
    
test=[1,3,200,100,1,3,200.01,200.001, 200.0001]
find_possible_matches(200,test)
    
    

(2, 200, 3)

In [12]:

match_list=[]
for i, row in df.iterrows():
    v=row[field_to_check]
    if v<0: #it is a credit, we search for debits that match
        try:      
            pos_deb, matched_value, matches_count =find_possible_matches(-v, debs_amt) #debs_amt.index(-v)
            if pos_deb==False:
                continue
            pos_cred=creds_pos.index(i)
            matched[debs_pos[pos_deb]]=True
            matched[i]=True
            matching_ref[debs_pos[pos_deb]]=[i]
            matching_ref[i]=[debs_pos[pos_deb]]
            match_list.append([[debs_pos[pos_deb]],[i]])
            debs_pos.pop(pos_deb)
            debs_amt.pop(pos_deb)         
            creds_pos.pop(pos_cred)
            creds_amt.pop(pos_cred)
        except Exception as e:
            if type(e).__name__ == 'ValueError':
                pass
            else:
                print(e)
                break
    else:
        try:
            pos_cred, matched_value, matches_count=find_possible_matches(-v, creds_amt) # creds_amt.index(-v)          
            if pos_cred==False:
                continue
            pos_deb=debs_pos.index(i)
            matched[i]=True
            matched[creds_pos[pos_cred]]=True
            matching_ref[creds_pos[pos_cred]]=[i]
            matching_ref[i]=[creds_pos[pos_cred]]
            match_list.append([[i],[creds_pos[pos_cred]]])
            debs_pos.pop(pos_deb)
            debs_amt.pop(pos_deb)         
            creds_pos.pop(pos_cred)
            creds_amt.pop(pos_cred)
            
        except Exception as e:
            if type(e).__name__ == 'ValueError':
                pass
            else:
                print(e)
                break
        

        
match_list[0:4]

[[[4], [3]], [[192], [30]], [[193], [31]], [[194], [32]]]

In [13]:
check_match_list()

1162 1162


In [14]:
print(len(debs_pos))
print(len(creds_pos))


1365
1065


In [15]:
from itertools import combinations
deb_perm_list=list(combinations(debs_pos,2))
cred_perm_list=list(combinations(creds_pos,2))


In [16]:
deb_perm_list_sum=[]
for x in deb_perm_list:
    try:   
        deb_perm_list_sum.append(round(list_to_check[x[0]]+list_to_check[x[1]],2))
    except:
        print("Error with ", x)
        break
cred_perm_list_sum=[]
for x in cred_perm_list:
    try:   
        cred_perm_list_sum.append(round(list_to_check[x[0]]+list_to_check[x[1]],2))
    except:
        print("Error with ", x)
        break


In [17]:
for i, row in df.iterrows():
    if matched[i]==False:
        v= row[field_to_check]
        if v<0:
            try:
                #here we search the credit in the list of combinations of debits taken by 2, summed
                pos_deb=deb_perm_list_sum.index(-v) 
                #if not found it raises a ValueError jumping to the exception: so we found something
                matched[i]=True
                matched[deb_perm_list[pos_deb][0]]=True
                matched[deb_perm_list[pos_deb][1]]=True            
                matching_ref[i]=deb_perm_list[pos_deb] # adds a tuple
                matching_ref[deb_perm_list[pos_deb][0]]= [pos_cred]
                matching_ref[deb_perm_list[pos_deb][1]]= [pos_cred]                 
                match_list.append([list(deb_perm_list[pos_deb]),[pos_cred]])
                
                #we remove the matched elements from the list of sums and their pointers 
                deb_perm_list_sum.pop(pos_deb)
                deb_perm_list.pop(pos_deb)
                
                debs_pointer1=debs_pos.index(deb_perm_list[pos_deb][0])
                debs_pos.pop(debs_pointer1)
                debs_amt.pop(debs_pointer1)
                
                debs_pointer2=debs_pos.index(deb_perm_list[pos_deb][1])
                debs_pos.pop(debs_pointer2)
                debs_amt.pop(debs_pointer2)
                
                creds_pointer=creds_pos.index(i)
                creds_pos.pop(creds_pointer)
                creds_amt.pop(creds_pointer)
                
                
                
            except Exception as e:
                if type(e).__name__ == 'ValueError':
                    pass
                else:
                    print(e)
                    break
                
        else:
            try:
                pos_cred=cred_perm_list_sum.index(-v) #a list/ pair!!!!
                pos_deb=i #this time the position is with respect to the main list   
                matched[pos_deb]=True
                matched[cred_perm_list[pos_cred][0]]=True
                matched[cred_perm_list[pos_cred][1]]=True         
                matching_ref[pos_deb]=cred_perm_list[pos_cred] 
                matching_ref[cred_perm_list[pos_cred][0]]= [pos_deb]
                matching_ref[cred_perm_list[pos_cred][1]]= [pos_deb]                
                match_list.append([[pos_deb], list(cred_perm_list[pos_cred])])
                cred_perm_list_sum.pop(pos_cred)
                cred_perm_list.pop(pos_cred)
                
                creds_pointer1=creds_pos.index(cred_perm_list[pos_cred][0])
                creds_pos.pop(creds_pointer1)
                creds_amt.pop(creds_pointer1)
                
                creds_pointer2=creds_pos.index(cred_perm_list[pos_cred][1])
                creds_pos.pop(creds_pointer2)
                creds_amt.pop(creds_pointer2)
                
                debs_pointer=debs_pos.index(i)
                debs_pos.pop(debs_pointer)
                debs_amt.pop(debs_pointer)
                
                
                
            except Exception as e:
                if type(e).__name__ == 'ValueError':
                    pass
                else:
                    print(e)
                    break

In [18]:
match_list[-20:]

[[[3384, 3443], [206438]],
 [[4241], [1615, 2359]],
 [[4246], [4244, 4245]],
 [[4250], [3302, 4572]],
 [[4287], [708, 2028]],
 [[4314], [3324, 4504]],
 [[4324], [175, 4542]],
 [[216, 396], [51947]],
 [[2220, 3487], [51947]],
 [[4385], [272, 497]],
 [[4448], [351, 3431]],
 [[4459], [638, 1038]],
 [[4483], [2092, 4365]],
 [[4491], [4489, 4490]],
 [[4513], [2200, 3806]],
 [[4538], [927, 2494]],
 [[4591], [878, 1626]],
 [[4616], [3279, 3280]],
 [[4714], [4712, 4713]],
 [[4519, 4520], [566447]]]

In [19]:
print("% pending to reconcile: ",(len(debs_amt)+len(creds_amt))/len(list_to_check))

% pending to reconcile:  0.3935633151030711
